In [1]:
from poisson_fem import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch

In [2]:
m = 2
mesh = PoissonFEM.RectangularMesh(np.ones(m)/m)
# mesh.plot()

In [3]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
def essBoundaryFun(x):
    return 1.0
mesh.set_essential_boundary(origin, essBoundaryFun)

In [4]:
def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domainBoundary)

In [5]:
#Define boundary flux field
def flux(x):
    a = np.array([1, 2, 3])
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [6]:
rhs = PoissonFEM.RightHandSide(mesh)

In [7]:
rhs.set_natural_rhs(mesh, flux)

In [8]:
K = PoissonFEM.StiffnessMatrix(mesh)

In [9]:
rhs.set_rhs_stencil(mesh, K)

In [10]:
lmbda_np = np.arange(5.0, m**2 + 5.0)
lmbda = PETSc.Vec().createSeq(mesh.n_cells)
rangeCells = range(mesh.n_cells)
lmbda.setValues(rangeCells, lmbda_np)

In [11]:
# Precomputations
x = PETSc.Vec().createSeq(mesh.n_eq)

In [12]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

In [13]:
N = int(1e3)

In [14]:
start = time.time()
for n in range(N):
    # Stiffness matrix assembly
    K.assemble(lmbda)
    # RHS assembly
    rhs.assemble(lmbda)
    # solving
    ksp.setOperators(K.matrix)
    ksp.solve(rhs.vector, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  1.2354850769042969e-05


In [15]:
# for scipy comparison
Kes = K.glob_stiff_stencil.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [16]:
start = time.time()
for n in range(N):
    Kvecs = Kes @ lmbda.array
    K1 = sps.csr_matrix((Kvecs[K.vec_nonzero], K.indices, K.indptr))
    rhs.assemble(lmbda)   # for fair comparison
    x1 = lg.spsolve(K1, rhs.vector.array)
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

scipy time =  0.00014408397674560547


In [17]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

result difference =  7.714025899500904e-16
PETSc speedup =  11.662138170590506


In [18]:
print(x1)

[1.11412457 1.20824635 1.22447663 1.43069946 1.60746189 1.38361105
 1.66285438 1.9079288 ]


# pytorch comparison

In [19]:
lmbda_torch = torch.tensor(lmbda_np, dtype=mesh.dtype).unsqueeze(0).expand(N, -1).unsqueeze(2)
rhs.expand_rhs_stencil_torch(N)

In [20]:
start = time.time()
K.assemble_torch(lmbda_torch)

# Assemble rhs
rhs.assemble_torch(lmbda_torch)

# solve
solution, LU = torch.solve(rhs.vector_torch, K.matrix_torch)
print('torch time == ', (time.time() - start)/N)

torch time ==  2.0389556884765624e-06


# autograd

In [21]:
lmbda_torch = torch.rand(N, mesh.n_cells, 1, dtype=mesh.dtype, requires_grad=True)

In [22]:
K.assemble_torch(lmbda_torch)

# Assemble rhs
rhs.assemble_torch(lmbda_torch)

# solve
solution, LU = torch.solve(rhs.vector_torch, K.matrix_torch)
print('torch time == ', (time.time() - start)/N)

torch time ==  1.619720458984375e-05


RuntimeError: grad can be implicitly created only for scalar outputs